In [5]:
from scipy.stats import chisquare
import numpy as np
import re

In [6]:
def metadata_dommap(f):
    file = open(f,"r")
    raw_data = file.readlines()[21:25]
    file.close()
    total = int(raw_data[0].rsplit()[-1])
    tot_nc = int(raw_data[1].rsplit()[-2])
    tot_cp = int(raw_data[2].rsplit()[-2])
    tot_is = int(raw_data[3].rsplit()[-2])
    tot_cnt = total - tot_nc - tot_cp - tot_is
    
    return {'NC': tot_nc, 'CP': tot_cp, 'IS': tot_is, 'CNT': tot_cnt, 'Total': total}

def read_dommap_tsv(f):
    file = open(f,"r")
    raw_data = file.readlines()[31:]
    file.close()
    data = [line.rsplit("\t")[:-1] for line in raw_data]
    return data

def count_X_grp_properties(data):
    X_grp_cnt = {}
    for d in data:
        uniprot = d[0].split("|")[1]
        prop = d[3]
        X_grp = d[5]
        F_grp = d[7]

        if X_grp not in X_grp_cnt.keys():
            X_grp_cnt[X_grp] = {"NC":0,"CP":0,"IS":0,"CNT":0,"Total":0,"ExpNC":0,"ExpCP":0,"ExpIS":0,"ExpCNT":0,
                                "NC_Fgrp":[],"CP_Fgrp":[],"IS_Fgrp":[]}

        if re.findall("NC",prop):
            X_grp_cnt[X_grp]["NC"] += 1
            X_grp_cnt[X_grp]["NC_Fgrp"].append([uniprot,F_grp])

        if re.findall("CP",prop):
            X_grp_cnt[X_grp]["CP"] += 1
            X_grp_cnt[X_grp]["CP_Fgrp"].append([uniprot,F_grp])

        if re.findall("IS",prop):
            X_grp_cnt[X_grp]["IS"] += 1
            X_grp_cnt[X_grp]["IS_Fgrp"].append([uniprot,F_grp])

        if not prop:
            X_grp_cnt[X_grp]["CNT"] += 1
        X_grp_cnt[X_grp]["Total"] += 1
        
    return X_grp_cnt

In [7]:
Mapped_Domains = [
"Yeast.mapped.tsv",
"Maize.mapped.tsv",
"FruitFly.mapped.tsv",
"Methano.mapped.tsv",
"Ecoli.mapped.tsv",
"Mouse.mapped.tsv",
"Human.mapped.tsv",
"Staph.mapped.tsv"
]

Organism = [name.split(".")[0] for name in Mapped_Domains]


In [8]:
Organism_X_grp_Count = {}
Organism_Tot_Dom_Count = {}
for MDom,Org in zip(Mapped_Domains, Organism):
    
    if Org not in Organism_X_grp_Count.keys():
        data = read_dommap_tsv(MDom)
        Organism_X_grp_Count[Org] = count_X_grp_properties(data)
         
    if Org not in Organism_Tot_Dom_Count.keys():
        Organism_Tot_Dom_Count[Org] = metadata_dommap(MDom)

In [9]:
np.seterr(invalid='ignore') # when it finds divide by 0 it will set to 0
Chi_Sq_Data = {}
for Org in Organism:
    # wfile = open("{}_Chi_Sq_Stats.tsv".format(Org),"w")
    # wfile.write("X-Group\tNC Count\tCP Count\tIS Count\tNormal Count\tRow Total\t\tEst. NC Count\tEst. CP Count\tEst. IS Count\tEst. Normal Count\t\tChi Square\tP-Value\t\tNC Obs/Exp\tCP Obs/Exp\tIS Obs/Exp\n")
    Chi_Sq_Data[Org] = {}

    Org_X_grp_Data = Organism_X_grp_Count[Org]
    Org_Tot_Data = Organism_Tot_Dom_Count[Org]
    
    exp_NC_ratio = (Org_Tot_Data["NC"]/Org_Tot_Data["Total"])
    exp_CP_ratio = (Org_Tot_Data["CP"]/Org_Tot_Data["Total"])
    exp_IS_ratio = (Org_Tot_Data["IS"]/Org_Tot_Data["Total"])
    exp_CNT_ratio = (Org_Tot_Data["CNT"]/Org_Tot_Data["Total"])

    for x_grp in Org_X_grp_Data.keys():
        obs_NC = Org_X_grp_Data[x_grp]["NC"]
        obs_CP = Org_X_grp_Data[x_grp]["CP"]
        obs_IS = Org_X_grp_Data[x_grp]["IS"]
        obs_CNT = Org_X_grp_Data[x_grp]["CNT"]
        
        tot = Org_X_grp_Data[x_grp]["Total"]
        exp_NC = tot*exp_NC_ratio
        exp_CP = tot*exp_CP_ratio
        exp_IS = tot*exp_IS_ratio
        exp_CNT = tot*exp_CNT_ratio
        
        Org_X_grp_Data[x_grp]["ExpNC"] = exp_NC
        Org_X_grp_Data[x_grp]["ExpCP"] = exp_CP
        Org_X_grp_Data[x_grp]["ExpIS"] = exp_IS
        Org_X_grp_Data[x_grp]["ExpCNT"] = exp_CNT

        if Org != "Staph":
            chi_data = chisquare([obs_NC,obs_CP,obs_IS,obs_CNT],[exp_NC,exp_CP,exp_IS,exp_CNT])
        else:
            chi_data = chisquare([obs_NC,obs_IS,obs_CNT],[exp_NC,exp_IS,exp_CNT])

        Chi_Sq_Data[Org][x_grp] = chi_data

        if exp_NC > 0:
            NC_oe = obs_NC/exp_NC
        else:
            NC_oe = 0

        if exp_CP > 0:
            CP_oe = obs_CP/exp_CP
        else:
            CP_oe = 0

        if exp_IS > 0:
            IS_oe = obs_IS/exp_IS
        else:
            IS_oe = 0
        
        write_str = "{}\t{}\t{}\t{}\t{}\t{}\t\t{}\t{}\t{}\t{}\t\t{}\t{}\t\t{}\t{}\t{}\n".format(x_grp,obs_NC,obs_CP,obs_IS,obs_CNT,tot,exp_NC,exp_CP,exp_IS,exp_CNT,chi_data.statistic,chi_data.pvalue,NC_oe,CP_oe,IS_oe)
    #     print(chisquare([obs_NC,obs_IS,obs_CNT],[exp_NC,exp_IS,exp_CNT]))
    #     wfile.write(write_str)
    
    # wfile.close()


In [10]:
# intresting_xgrp = ["NAP-like","Class II aaRS and biotin synthetases","HUP domain-like", "Alkaline phosphatase-like","DCoH-like","Anticodon-binding domain of a subclass of class I aminoacyl-tRNA synthetases", "TIM beta/alpha-barrel", "triple barrel", "Cysteine proteinases-like"]
intresting_xgrp = ["Neurotransmitter-gated ion-channel transmembrane pore", "Proton glutamate symport protein", "dicarboxylate/sodium symporter","Sodium/Calcium exchanger","Voltage-gated ion channels","Sodium:neurotransmitter symporter family (SNF)-like"]
# intresting_xgrp = ["TIM beta/alpha-barrel"]

# intresting_xgrp = ["Metal cation-transporting ATPase, ATP-binding domain",
# "DnaJ/Hsp40 cysteine-rich domain",
# "FAD-linked reductases, C-terminal domain-like",
# "Rubredoxin-like",
# "FKBP-like",
# "cradle loop barrel",
# "Flavodoxin-like"]

# intresting_xgrp=["Glycogen debranching enzyme (GDE) insertion domain",
# "NADPH-cytochrome p450 reductase helical insertion domain",
# "tRNA pseudouridine synthase TruD insertion domain",
# "Prokaryotic AspRS, insert domain",
# "Insertion subdomain in DsbA-like",
# "insertion domain in beta subunit of DNA dependent RNA-polymerase",
# "Transducin (alpha subunit), insertion domain"]

# intresting_xgrp = ["Rubredoxin-like"]

Org = "Human"
# for Org in Chi_Sq_Data.keys():
#     print(Org,"::")

for xgrp in intresting_xgrp:
    print(xgrp,"::")
    Chi_Stat = Chi_Sq_Data[Org][xgrp]
    chi_val, p_val = Chi_Stat

    Prop_Count = Organism_X_grp_Count[Org][xgrp]
    nc_cnt, cp_cnt, is_cnt, norm_cnt, tot, nc_est, cp_est, is_est, norm_est = [Prop_Count[p] for p in Prop_Count][:9]
    print(sum([nc_cnt/tot, cp_cnt/tot, is_cnt/tot, norm_cnt/tot]))
    print(f"{100*nc_cnt/tot:3.2f}, {100*cp_cnt/tot:3.2f}, {100*is_cnt/tot:3.2f}, {100*norm_cnt/tot:3.2f}")
    # print(f"{100*is_cnt/tot:3.2f}, {100*nc_cnt/tot:3.2f}, {100*cp_cnt/tot:3.2f}, {100*norm_cnt/tot:3.2f}")
    print(tot, "/",f"{Chi_Stat.pvalue:3.1E}")
    # print(nc_cnt, cp_cnt, is_cnt, norm_cnt, tot, 0,nc_est, cp_est, is_est, norm_est, 0,chi_val, p_val, nc_cnt/nc_est if nc_est > 0 else 0, cp_cnt/cp_est if cp_est > 0 else 0, is_cnt/is_est if is_est > 0 else 0)
    # print(("{}\t"*11).format(*[Prop_Count[p] for p in Prop_Count],Chi_Stat.statistic,Chi_Stat.pvalue))
    # print(100*np.array([Prop_Count[p] for p in Prop_Count][0:4])/sum([Prop_Count[p] for p in Prop_Count][0:4]))

Neurotransmitter-gated ion-channel transmembrane pore ::
1.0
75.82, 0.00, 0.00, 24.18
153 / 0.0E+00
Proton glutamate symport protein ::
1.0
57.14, 0.00, 0.00, 42.86
63 / 2.3E-87
dicarboxylate/sodium symporter ::
1.0
45.95, 0.00, 0.00, 54.05
37 / 9.6E-32
Sodium/Calcium exchanger ::
1.1428571428571428
20.63, 14.29, 0.00, 79.37
63 / 2.8E-73
Voltage-gated ion channels ::
1.0214909335124245
36.27, 2.08, 5.51, 58.29
1489 / 0.0E+00
Sodium:neurotransmitter symporter family (SNF)-like ::
1.0
22.78, 0.00, 0.00, 77.22
259 / 6.4E-44


In [11]:
# Finds the NC F_grp that the IS F_grp of a given X_grp is within

hum_data = read_dommap_tsv("Human.mapped.tsv")

unique_IS_f_grp = [] # gives all insertional f groups without repeats
unique_NC_f_grp = [] # gives all host f groups without repeats
pair_f_grp = [] # gives pairs in [IS F group, NC (host) F group] for all insertional domains
NC_group_IS_pairs = {} # gives a dict with NC f group keys and values with all IS f groups that insert into it, better organized than pairs
for pair in Organism_X_grp_Count["Human"]["P-loop domains-like"]["IS_Fgrp"]:
    
    for query in hum_data:
        if pair[0] in query[0] and "NC" in query[3]:
            nc_fgrp = query[7]
            if nc_fgrp not in unique_NC_f_grp:
                unique_NC_f_grp.append(nc_fgrp)
            if [pair[1],nc_fgrp] not in pair_f_grp:
                pair_f_grp.append([pair[1],nc_fgrp])
            if nc_fgrp not in NC_group_IS_pairs.keys():
                NC_group_IS_pairs[nc_fgrp] = []
            if pair[1] not in NC_group_IS_pairs[nc_fgrp]:
                NC_group_IS_pairs[nc_fgrp].append(pair[1])
            

    if pair[1] not in unique_IS_f_grp:
        unique_IS_f_grp.append(pair[1])


print(unique_IS_f_grp)

print(unique_NC_f_grp)

print(pair_f_grp,"\n")

print(NC_group_IS_pairs,"\n")

['SRP54', 'ABC_tran_1', 'Septin', 'Beta-Casp', 'CLP1_P', 'MnmE_helical_2nd', 'TIP49_1st', 'Sigma54_activat']
['AAA_33', 'ABC_membrane', 'Sigma54_activat', 'Lactamase_B_6', 'MnmE_helical_2nd', 'Fzo_mitofusin', 'AAA_7_N', 'AAA_19']
[['SRP54', 'AAA_33'], ['ABC_tran_1', 'ABC_membrane'], ['Septin', 'Sigma54_activat'], ['Beta-Casp', 'Lactamase_B_6'], ['CLP1_P', 'MnmE_helical_2nd'], ['MnmE_helical_2nd', 'Fzo_mitofusin'], ['MnmE_helical_2nd', 'MnmE_helical_2nd'], ['TIP49_1st', 'AAA_7_N'], ['Sigma54_activat', 'AAA_19']] 

{'AAA_33': ['SRP54'], 'ABC_membrane': ['ABC_tran_1'], 'Sigma54_activat': ['Septin'], 'Lactamase_B_6': ['Beta-Casp'], 'MnmE_helical_2nd': ['CLP1_P', 'MnmE_helical_2nd'], 'Fzo_mitofusin': ['MnmE_helical_2nd'], 'AAA_7_N': ['TIP49_1st'], 'AAA_19': ['Sigma54_activat']} 



In [19]:
for org, data in Organism_Tot_Dom_Count.items():
    nc = data["NC"]
    cp = data["CP"]
    iss = data["IS"]
    tot = data["Total"]
    cnt = tot - (nc + cp + iss)
    pNC = data["NC"]/data["Total"]
    pCP = data["CP"]/data["Total"]
    pIS = data["IS"]/data["Total"]
    pCON = data["CNT"]/data["Total"]
    
    print(data["Total"])
    print(org, f"N = {tot:,} (NC: {pNC:2.2%}, CP: {pCP:2.2%}, IS: {pIS:2.2%}, CON: {pCON:2.2%})")
    
    print(f"NC {nc:,} CP {cp:,} IS {iss:,} CON {cnt:,}")

    print(f"NC/IS : {nc/iss:3.3f}")

7310
Yeast N = 7,310 (NC: 10.74%, CP: 0.26%, IS: 2.63%, CON: 86.37%)
NC 785 CP 19 IS 192 CON 6,314
NC/IS : 4.089
119081
Maize N = 119,081 (NC: 8.95%, CP: 0.53%, IS: 2.18%, CON: 88.34%)
NC 10,654 CP 631 IS 2,598 CON 105,198
NC/IS : 4.101
44432
FruitFly N = 44,432 (NC: 6.13%, CP: 0.33%, IS: 2.44%, CON: 91.10%)
NC 2,722 CP 146 IS 1,086 CON 40,478
NC/IS : 2.506
2256
Methano N = 2,256 (NC: 6.47%, CP: 0.44%, IS: 4.26%, CON: 88.83%)
NC 146 CP 10 IS 96 CON 2,004
NC/IS : 1.521
6162
Ecoli N = 6,162 (NC: 4.98%, CP: 0.13%, IS: 2.82%, CON: 92.06%)
NC 307 CP 8 IS 174 CON 5,673
NC/IS : 1.764
104048
Mouse N = 104,048 (NC: 4.57%, CP: 0.35%, IS: 2.74%, CON: 92.35%)
NC 4,752 CP 359 IS 2,848 CON 96,089
NC/IS : 1.669
155158
Human N = 155,158 (NC: 4.52%, CP: 0.40%, IS: 1.72%, CON: 93.36%)
NC 7,013 CP 626 IS 2,661 CON 144,858
NC/IS : 2.635
3582
Staph N = 3,582 (NC: 4.05%, CP: 0.14%, IS: 3.85%, CON: 91.96%)
NC 145 CP 5 IS 138 CON 3,294
NC/IS : 1.051
